In [1]:
import requests
import json

URL = 'http://0.0.0.0:5000'
START_PORT = 55000

* Ejecutamos OaaS Node y comprobamos que podemos acceder a su API: <http://0.0.0.0:5000> 
* Partimos de que tenemos subido el algoritmo al registry de Optimaix
* El algorithm esta subido al registry con el nombre: _optimaix/micro-nfplaning_

### 1. Dar del alta el Repository de Algorithm dentro de OaaS Node

In [2]:
# Comprobar que repositorios hay actualmente en la plataforma

# GET /images
response = requests.get(URL + '/images')

if response.status_code == 200:
    data = response.json()  # Parse JSON data into a dictionary
    print(json.dumps(data, indent=4))  # Pretty-print the JSON data
else:
    print("Failed to retrieve data")

{
    "items": [
        {
            "id": 10,
            "name": "string1",
            "description": "string",
            "url": "algorithm_repo",
            "type": "python",
            "algorithms": [
                {
                    "algorithmId": 11,
                    "localId": 0,
                    "name": "ExampleAlgorithm",
                    "version": "0.1",
                    "execution_type": "OaaS",
                    "description": "Example Algorithm",
                    "inputParametersTemplate": {},
                    "outputParametersTemplateSuccess": {},
                    "outputParametersTemplateFailure": null
                },
                {
                    "algorithmId": 12,
                    "localId": 1,
                    "name": "TestAlgorithm",
                    "version": "0.2",
                    "execution_type": "OaaS",
                    "description": "Example Algorithm 2",
                    "inputParametersTempla

In [12]:
# Damos de alta el NDT como repositorio de tipo general (no algoritmos inside)

req = {
    "name": "EDGE Planning",
    "description": "Algorithm repository for EDGE Planning",
    "url": "optimaix/micro-nfplaning",
    "type": "matlab"
}

print(req)

# Sent a POST to /algorithmRepository
response = requests.post(URL + '/images', json=req)

if response.status_code == 200 or response.status_code == 201:
    data = response.json()  # Parse JSON data into a dictionary
    print(json.dumps(data, indent=4))  # Pretty-print the JSON data
else:
    data = response.json()
    print(json.dumps(data, indent=4))
    print("Failed to retrieve data")

{'name': 'EDGE Planning', 'description': 'Algorithm repository for EDGE Planning', 'url': 'optimaix/micro-nfplaning', 'type': 'matlab'}
{
    "id": 39,
    "name": "EDGE Planning",
    "description": "Algorithm repository for EDGE Planning",
    "url": "optimaix/micro-nfplaning",
    "type": "matlab",
    "algorithms": [
        {
            "algorithmId": 20,
            "localId": 0,
            "name": "optimaix/micro-nfplaning",
            "version": "1.0",
            "execution_type": "direct",
            "description": "MATLAB Runtime for algorithm optimaix/micro-nfplaning",
            "inputParametersTemplate": {},
            "outputParametersTemplateSuccess": {},
            "outputParametersTemplateFailure": null
        }
    ],
    "instances": []
}


### 2. Instanciar algoritmo dentro de plataforma OaaS

In [13]:
repository_algorithm_id = 39

In [14]:
req = {
    "name": "Algorithm edge planning use case",
    "description": "algorithm for edge planning use case",
    "quotaInformation": "none",
    "image_id": repository_algorithm_id
}

print(req)

# Sent a POST to /algorithmRepository
response = requests.post(URL + '/instances', json=req)

if response.status_code == 200 or response.status_code == 201:
    data = response.json()  # Parse JSON data into a dictionary
    print(json.dumps(data, indent=4))  # Pretty-print the JSON data
else:
    data = response.json()
    print(json.dumps(data, indent=4))
    print("Failed to retrieve data")

{'name': 'Algorithm edge planning use case', 'description': 'algorithm for edge planning use case', 'quotaInformation': 'none', 'repository_id': 39}
{
    "id": 5,
    "name": "Algorithm edge planning use case",
    "description": "algorithm for edge planning use case",
    "status": "running",
    "quotaInformation": "none",
    "repository": {
        "id": 39,
        "name": "EDGE Planning",
        "description": "Algorithm repository for EDGE Planning",
        "url": "optimaix/micro-nfplaning",
        "type": "matlab"
    }
}


In [4]:
instance_id = 5

## 3. Execute algorithm

In [22]:
endpoint = "NFPSarchive/NFPlanningSolverStructs"
algorithm_URL = URL + f'/instances/{instance_id}/execute/{endpoint}'

with open('algoritmo-javier/input_algoritmo_javier.json', 'r') as file:
    data = json.load(file)
    

#print(json.dumps(json.loads(data['rhs'][0]), indent=4))

In [19]:
headers = {
    'Content-Type': 'application/json'
}

# Send the POST request
response = requests.post(algorithm_URL, headers=headers, json=data)

# Print the response from the server
print(response.text)

{"lhs":["{   \"comment\": \"EDGE problem planning, Version 1 202403311716 Exp=1, ct=5, st=5, instname := d_1_3_3\",   \"network\":     {       \"elements\": [         {           \"name\": \"inf1\",           \"microservices\": [             {               \"service_name\":\"serv1\",               \"microservice_name\":\"micrServ1\"             },             {               \"service_name\":\"serv1\",               \"microservice_name\":\"micrServ2\"             },             {               \"service_name\":\"serv2\",               \"microservice_name\":\"micrServ1\"             }           ]         },         {           \"name\": \"inf1\",           \"microservices\": [             {               \"service_name\":\"serv3\",               \"microservice_name\":\"micrServ1\"             },             {               \"service_name\":\"serv3\",               \"microservice_name\":\"micrServ2\"             }           ]         }        ]     } }"]}


In [7]:
# Process the response as JSON
response_data = response.json()
output = json.loads(response_data["lhs"][0])
pretty_json = json.dumps(output, indent=4, sort_keys=True)
print(pretty_json)

{
    "comment": "EDGE problem planning, Version 1 202403311716 Exp=1, ct=5, st=5, instname := d_1_3_3",
    "network": {
        "elements": [
            {
                "microservices": [
                    {
                        "microservice_name": "micrServ1",
                        "service_name": "serv1"
                    },
                    {
                        "microservice_name": "micrServ2",
                        "service_name": "serv1"
                    },
                    {
                        "microservice_name": "micrServ1",
                        "service_name": "serv2"
                    }
                ],
                "name": "inf1"
            },
            {
                "microservices": [
                    {
                        "microservice_name": "micrServ1",
                        "service_name": "serv3"
                    },
                    {
                        "microservice_name": "micrServ2",
          

# 4. 

In [25]:
# Stop Instance of NDT
response = requests.delete(URL + f'/instances/{instance_id}/stop')

if response.status_code == 200 or response.status_code == 201:
    data = response.json()  # Parse JSON data into a dictionary
    print(json.dumps(data, indent=4))  # Pretty-print the JSON data
else:
    data = response.json()
    print(json.dumps(data, indent=4))
    print("Failed to retrieve data")

{
    "message": "Stopped instance 5"
}
